#🧠 Agent Development with ToolCalling & ReAct Agents using LangGraph

#LLM Purpose: Uses Gemini Flash API
Concepts Covered:

ReAct Pattern (Reasoning + Acting)

Tool-Calling Integration

Stateful LangGraph Agent Execution

#✅ Scenario:
We’ll build a LangGraph ReAct Agent that:

1.Receives a user question.

2.Decides to either:

    Call a calculator tool (if math is needed), OR

    Use Gemini to answer it directly.

3.Returns the final response.

#✅ Step-by-Step Agent with Gemini + LangGraph + Tool-Calling
#🔧 Prerequisites

In [1]:
!pip install langgraph google-generativeai

#✅ Step-by-Step Implementation
#Step 1: Imports library & Step 2: Gemini Setup

In [ ]:
# Step 1: Import necessary libraries
import os
import re
from typing import TypedDict, Literal
import google.generativeai as genai
from langgraph.graph import StateGraph, END
from google.colab import userdata # Import userdata for Colab secrets


# Step 2: Set up Gemini API key
# To use the Gemini API, you'll need an API key. If you don't already have one,
# create a key in Google AI Studio. In Colab, add the key to the secrets manager
# under the "🔑" in the left panel. Give it the name `GOOGLE_API_KEY`.
# Then pass the key to the SDK:
os.environ["GOOGLE_API_KEY"] = userdata.get("GOOGLE_API_KEY")
genai.configure(api_key=os.environ["GOOGLE_API_KEY"])

#📄 Step 3: Define Agent State Schema

In [ ]:
class AgentState(TypedDict):
    question: str
    thought: str
    tool_name: str
    tool_input: str
    tool_output: str
    final_answer: str


#🛠️ Step 4: Define Tool — Calculator

In [ ]:
def calculator_tool(expression: str) -> str:
    try:
        # Very basic expression evaluator — safe
        result = eval(re.sub(r"[^0-9+\-*/(). ]", "", expression))
        return str(result)
    except Exception as e:
        return f"Error in calculation: {e}"


#🤖 Step 5: Gemini-Powered ReAct Agent Parser

In [ ]:
# Define a function to parse ReAct-style responses from Gemini
# It returns a structured dictionary containing reasoning, tool usage, and final answer
def gemini_react_parser(question: str) -> dict:

    # Initialize the Gemini Flash model for fast reasoning and response generation
    model = genai.GenerativeModel("models/gemini-flash-latest")

    # Construct a ReAct-style prompt
    # The prompt instructs the model to reason step-by-step and optionally use a tool
    react_prompt = f"""Answer the following question using the ReAct pattern.

Question: {question}

Think step-by-step and decide if you need to use a tool. Available tool: "Calculator".
If needed, respond using this format:

Thought: [your reasoning]
Tool: [Calculator]
Tool Input: [math expression]

Otherwise, respond with:

Thought: [your reasoning]
Final Answer: [your direct answer]"""

    # Generate content using the Gemini model
    response = model.generate_content(react_prompt)

    # Extract and clean the textual response from the model
    text = response.text.strip()

    # Use regular expressions to extract different ReAct components
    # These patterns look for labeled sections in the LLM output
    thought = re.search(r"Thought:\s*(.*)", text)
    tool = re.search(r"Tool:\s*(.*)", text)
    tool_input = re.search(r"Tool Input:\s*(.*)", text)
    final_answer = re.search(r"Final Answer:\s*(.*)", text)

    # Return a structured dictionary with safe fallbacks if patterns are not found
    return {
        # Model's reasoning or thought process
        "thought": thought.group(1) if thought else "",

        # Name of the tool requested by the model (e.g., Calculator)
        "tool_name": tool.group(1).strip() if tool else "",

        # Input passed to the tool (e.g., mathematical expression)
        "tool_input": tool_input.group(1).strip() if tool_input else "",

        # Final answer generated by the model
        "final_answer": final_answer.group(1).strip() if final_answer else ""
    }

#🧩 Step 6: LangGraph Nodes
🔹 Node 1: ReAct Reasoner (LLM Parser)

In [ ]:
def react_reason_node(state: AgentState) -> AgentState:
    parsed = gemini_react_parser(state["question"])
    return {**state, **parsed}


🔹 Node 2: Tool-Caller Node

In [ ]:
def tool_call_node(state: AgentState) -> AgentState:
    output = calculator_tool(state["tool_input"])
    return {**state, "tool_output": output}


🔹 Node 3: Final Answer Generator (if tool was used)

In [ ]:
# Define the final node in a ReAct-style agent workflow
# This node is responsible for producing the final answer
def react_final_node(state: AgentState) -> AgentState:

    # Check if the agent decided to use a tool in earlier steps
    if state["tool_name"]:

        # Build a prompt that asks Gemini to synthesize the final answer
        # using the original question and the tool execution result
        prompt = f"""You were asked: {state['question']}
You used tool {state['tool_name']} with input {state['tool_input']} and got result {state['tool_output']}.

Now provide the final answer clearly."""

        # Initialize the Gemini Flash model for fast response generation
        model = genai.GenerativeModel("models/gemini-flash-latest")

        # Generate the final response from Gemini
        response = model.generate_content(prompt)

        # Update the agent state with the final answer
        # Preserve all existing state values using dictionary unpacking
        return {
            **state,
            "final_answer": response.text.strip()
        }

    else:
        # If no tool was used, the final answer is already present in the state
        # Return the state unchanged
        return state


#🔁 Step 7: Define Conditional Route

In [ ]:
def tool_needed_router(state: AgentState) -> str:
    return "tool_call" if state["tool_name"] else "react_final"


#🏗️ Step 8: Build LangGraph Agent

In [ ]:
builder = StateGraph(AgentState)

# Add nodes
builder.add_node("react_reason", react_reason_node)
builder.add_node("tool_call", tool_call_node)
builder.add_node("react_final", react_final_node)

# Routing logic
builder.set_entry_point("react_reason")
builder.add_conditional_edges("react_reason", tool_needed_router)
builder.add_edge("tool_call", "react_final")
builder.add_edge("react_final", END)

# Compile
graph = builder.compile()


#▶️ Step 9: Run the Agent

In [ ]:
# Try question with math
input_1 = {"question": "What is 25 * 12 + 100?"}
output_1 = graph.invoke(input_1)

# Try question without math
input_2 = {"question": "What is the capital of Japan?"}
output_2 = graph.invoke(input_2)

# Display Results
print("🔍 Question:", output_1["question"])
print("💭 Thought:", output_1["thought"])
print("🛠️ Tool Used:", output_1["tool_name"])
print("🧮 Tool Output:", output_1["tool_output"])
print("✅ Final Answer:", output_1["final_answer"])
print("—" * 50)
print("🔍 Question:", output_2["question"])
print("💭 Thought:", output_2["thought"])
print("🛠️ Tool Used:", output_2["tool_name"])
print("✅ Final Answer:", output_2["final_answer"])

#🧠 Summary of What You Learned

| Component                    | Description                                            |
| ---------------------------- | ------------------------------------------------------ |
| **ReAct Pattern**            | Combined reasoning + action via Gemini                 |
| **Tool Calling**             | Dynamically invokes calculator if needed               |
| **Stateful LangGraph Agent** | Built a graph that conditionally routes logic          |
| **Gemini API**               | Used Gemini 1.5 Flash for reasoning and final response |
